In [7]:
import re 
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException, WebDriverException
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 구글 맵 페이지 열기 및 '뱅뱅막국수' 검색
driver.get("https://www.google.com/maps")
try:
    # 검색어 입력 및 검색
    search_box = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="searchboxinput"]'))
    )
    search_box.send_keys("스노우폭스 뱅뱅점")
    search_box.send_keys(Keys.ENTER)
except Exception as e:
    print(f"검색창을 찾을 수 없습니다: {e}")
    driver.quit()
    exit()

# 페이지 로드 대기
time.sleep(5)

In [8]:
try:
    # 첫 번째 검색 결과를 JavaScript로 클릭
    first_result = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '(//*[@class="hfpxzc"])[1]'))
    )
    driver.execute_script("arguments[0].click();", first_result)
except Exception as e:
    print(f"첫 번째 검색 결과를 클릭할 수 없습니다: {e}")
    



첫 번째 검색 결과를 클릭할 수 없습니다: Message: 
Stacktrace:
	GetHandleVerifier [0x00BFEBD3+24307]
	(No symbol) [0x00B88D74]
	(No symbol) [0x00A6C323]
	(No symbol) [0x00AADC86]
	(No symbol) [0x00AADECB]
	(No symbol) [0x00AEB9F2]
	(No symbol) [0x00ACFED4]
	(No symbol) [0x00AE9579]
	(No symbol) [0x00ACFC26]
	(No symbol) [0x00AA219C]
	(No symbol) [0x00AA311D]
	GetHandleVerifier [0x00EA8D93+2818227]
	GetHandleVerifier [0x00F0542E+3196750]
	GetHandleVerifier [0x00EFD9D2+3165426]
	GetHandleVerifier [0x00C9DA70+675216]
	(No symbol) [0x00B91B3D]
	(No symbol) [0x00B8EA18]
	(No symbol) [0x00B8EBB5]
	(No symbol) [0x00B81640]
	BaseThreadInitThunk [0x758DFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x7787809E+286]
	RtlGetAppContainerNamedObjectPath [0x7787806E+238]



In [9]:
# 리뷰 페이지 열기
try:
    review_button = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.XPATH, '//button[contains(@aria-label, "리뷰 더보기")]'))
    )
    review_button.click()
    time.sleep(5)  # 리뷰 페이지가 완전히 로드되도록 추가 대기
except Exception as e:
    print(f"리뷰 페이지를 열 수 없습니다: {e}")
    driver.quit()
    exit()

In [10]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

# 리뷰 섹션의 컨테이너를 찾음
try:
    review_container = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div:nth-child(2) > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde'))
    ) #목록이 하나가 아닌경우: #QA0Szd > div > div > div.w6VYqd > div.bJzME.Hu9e2e.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde
    
    # 스크롤을 반복적으로 내리기
    max_wait_time = 5  # 최대 대기 시간 시작 (초)
    while True:
        # 현재 로드된 리뷰 수 확인
        reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')
        previous_count = len(reviews)
        
        # 스크롤을 아래로 내림
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", review_container)
        
        # 새로운 리뷰가 로드될 때까지 대기
        try:
            WebDriverWait(driver, max_wait_time).until(
                lambda driver: len(driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')) > previous_count
            )
            # 리뷰가 로드되면 대기 시간을 초기화
            max_wait_time = 5
        except TimeoutException:
            # 새로운 리뷰가 로드되지 않으면 더 이상 로드할 리뷰가 없는 것으로 간주하고 종료
            print("더 이상 로드할 리뷰가 없습니다.")
            break

        # 스크롤이 아래로 내려갈수록 로딩 시간이 길어질 수 있으므로, 실패 시 대기 시간 증가
        max_wait_time += 1  # 한 번 실패할 때마다 대기 시간을 1초씩 늘림 (필요 시 조정 가능)

    print("모든 리뷰를 끝까지 로드했습니다.")
except Exception as e:
    print(f"스크롤 중 오류 발생: {e}")


더 이상 로드할 리뷰가 없습니다.
모든 리뷰를 끝까지 로드했습니다.


In [11]:

# 리뷰 데이터 수집
reviews_list = []
reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')
for review in reviews:
    try:
        # '자세히' 버튼 클릭
        try:
            more_button = review.find_element(By.XPATH, './/span[2]/button')
            more_button.click()
            time.sleep(1)  # '자세히' 클릭 후 리뷰 로드 대기
        except NoSuchElementException:
            pass  # '자세히' 버튼이 없으면 무시하고 진행
        
        # 리뷰 작성자 ID
        reviewer_id = review.find_element(By.CSS_SELECTOR, '.d4r55').text
        # 리뷰 작성 날짜
        date = review.find_element(By.CSS_SELECTOR, '.rsqaWe').text
        # 리뷰 내용
        review_text = review.find_element(By.CSS_SELECTOR, '.wiI7pd').text
        reviews_list.append({
            "ID": reviewer_id,
            "Date": date,
            "Review": review_text
        })
    except Exception as e:
        print(f"리뷰 수집 중 오류: {e}")
        continue

# 데이터프레임으로 변환 및 출력
reviews_df = pd.DataFrame(reviews_list)
print(reviews_df)

# CSV 파일로 저장
#reviews_df.to_csv('bangbang_makguksu_reviews.csv', index=False, encoding='utf-8-sig')

# 드라이버 종료
#driver.quit()

리뷰 수집 중 오류: Message: no such element: Unable to locate element: {"method":"css selector","selector":".wiI7pd"}
  (Session info: chrome=130.0.6723.119); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00BFEBD3+24307]
	(No symbol) [0x00B88D74]
	(No symbol) [0x00A6C323]
	(No symbol) [0x00AADC86]
	(No symbol) [0x00AADECB]
	(No symbol) [0x00AA3CF1]
	(No symbol) [0x00ACFED4]
	(No symbol) [0x00AA3C14]
	(No symbol) [0x00AD0124]
	(No symbol) [0x00AE9579]
	(No symbol) [0x00ACFC26]
	(No symbol) [0x00AA219C]
	(No symbol) [0x00AA311D]
	GetHandleVerifier [0x00EA8D93+2818227]
	GetHandleVerifier [0x00F0542E+3196750]
	GetHandleVerifier [0x00EFD9D2+3165426]
	GetHandleVerifier [0x00C9DA70+675216]
	(No symbol) [0x00B91B3D]
	(No symbol) [0x00B8EA18]
	(No symbol) [0x00B8EBB5]
	(No symbol) [0x00B81640]
	BaseThreadInitThunk [0x758DFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x778

리뷰 수집 중 오류: Message: no such element: Unable to locate element: {"method":"css selector","selector":".wiI7pd"}
  (Session info: chrome=130.0.6723.119); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00BFEBD3+24307]
	(No symbol) [0x00B88D74]
	(No symbol) [0x00A6C323]
	(No symbol) [0x00AADC86]
	(No symbol) [0x00AADECB]
	(No symbol) [0x00AA3CF1]
	(No symbol) [0x00ACFED4]
	(No symbol) [0x00AA3C14]
	(No symbol) [0x00AD0124]
	(No symbol) [0x00AE9579]
	(No symbol) [0x00ACFC26]
	(No symbol) [0x00AA219C]
	(No symbol) [0x00AA311D]
	GetHandleVerifier [0x00EA8D93+2818227]
	GetHandleVerifier [0x00F0542E+3196750]
	GetHandleVerifier [0x00EFD9D2+3165426]
	GetHandleVerifier [0x00C9DA70+675216]
	(No symbol) [0x00B91B3D]
	(No symbol) [0x00B8EA18]
	(No symbol) [0x00B8EBB5]
	(No symbol) [0x00B81640]
	BaseThreadInitThunk [0x758DFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x778

리뷰 수집 중 오류: Message: no such element: Unable to locate element: {"method":"css selector","selector":".wiI7pd"}
  (Session info: chrome=130.0.6723.119); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00BFEBD3+24307]
	(No symbol) [0x00B88D74]
	(No symbol) [0x00A6C323]
	(No symbol) [0x00AADC86]
	(No symbol) [0x00AADECB]
	(No symbol) [0x00AA3CF1]
	(No symbol) [0x00ACFED4]
	(No symbol) [0x00AA3C14]
	(No symbol) [0x00AD0124]
	(No symbol) [0x00AE9579]
	(No symbol) [0x00ACFC26]
	(No symbol) [0x00AA219C]
	(No symbol) [0x00AA311D]
	GetHandleVerifier [0x00EA8D93+2818227]
	GetHandleVerifier [0x00F0542E+3196750]
	GetHandleVerifier [0x00EFD9D2+3165426]
	GetHandleVerifier [0x00C9DA70+675216]
	(No symbol) [0x00B91B3D]
	(No symbol) [0x00B8EA18]
	(No symbol) [0x00B8EBB5]
	(No symbol) [0x00B81640]
	BaseThreadInitThunk [0x758DFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x778

리뷰 수집 중 오류: Message: no such element: Unable to locate element: {"method":"css selector","selector":".wiI7pd"}
  (Session info: chrome=130.0.6723.119); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00BFEBD3+24307]
	(No symbol) [0x00B88D74]
	(No symbol) [0x00A6C323]
	(No symbol) [0x00AADC86]
	(No symbol) [0x00AADECB]
	(No symbol) [0x00AA3CF1]
	(No symbol) [0x00ACFED4]
	(No symbol) [0x00AA3C14]
	(No symbol) [0x00AD0124]
	(No symbol) [0x00AE9579]
	(No symbol) [0x00ACFC26]
	(No symbol) [0x00AA219C]
	(No symbol) [0x00AA311D]
	GetHandleVerifier [0x00EA8D93+2818227]
	GetHandleVerifier [0x00F0542E+3196750]
	GetHandleVerifier [0x00EFD9D2+3165426]
	GetHandleVerifier [0x00C9DA70+675216]
	(No symbol) [0x00B91B3D]
	(No symbol) [0x00B8EA18]
	(No symbol) [0x00B8EBB5]
	(No symbol) [0x00B81640]
	BaseThreadInitThunk [0x758DFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x778

리뷰 수집 중 오류: Message: no such element: Unable to locate element: {"method":"css selector","selector":".wiI7pd"}
  (Session info: chrome=130.0.6723.119); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00BFEBD3+24307]
	(No symbol) [0x00B88D74]
	(No symbol) [0x00A6C323]
	(No symbol) [0x00AADC86]
	(No symbol) [0x00AADECB]
	(No symbol) [0x00AA3CF1]
	(No symbol) [0x00ACFED4]
	(No symbol) [0x00AA3C14]
	(No symbol) [0x00AD0124]
	(No symbol) [0x00AE9579]
	(No symbol) [0x00ACFC26]
	(No symbol) [0x00AA219C]
	(No symbol) [0x00AA311D]
	GetHandleVerifier [0x00EA8D93+2818227]
	GetHandleVerifier [0x00F0542E+3196750]
	GetHandleVerifier [0x00EFD9D2+3165426]
	GetHandleVerifier [0x00C9DA70+675216]
	(No symbol) [0x00B91B3D]
	(No symbol) [0x00B8EA18]
	(No symbol) [0x00B8EBB5]
	(No symbol) [0x00B81640]
	BaseThreadInitThunk [0x758DFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x778

In [12]:
reviews_df

,ID,Date,Review
0,이정민,10달 전,●접근성\n뱅뱅사거리 버스정류장에서 3분 이내\n\n●주차\n건물 지하 이용\n진입...
1,이샤샤,1년 전,SNOW FOX(feat.장문녀)\n\n롤이 땡긴다싶어서 롤전문점을 찾아보다가 알게...
2,S K,6년 전,진작에 여기 평점을 봤어야 됬는데..\n\n맛 자체는 나쁘지 않다. 깔끔하고 도시락...
3,머니 바이블,5년 전,깔끔하고 괜찮아요. 도시락치곤 가격 좀 센데 재료 궐리티가 높다고 하니 가끔 먹기엔...
4,Ryung-Hwa Rhee,5년 전,가격이 좀 비싸지만 혼밥하기 좋은 밥집이에요\n건강한 메뉴들로 먹고 싶을때 자주 먹...
5,Irene “MJ” Kim,5년 전,가격은 어느 근사한 음식점만큼 비싸다. 하지만 청결한 매장에서 신선한 식재료로 조리...
6,Sophia Ryu,5년 전,가격은 좀 나가지만 맛은 좋아요. 혼자 와서 간단히 샐러드나 롤 먹고 가기 좋을 듯해요.
7,Minji Kim,3년 전,가격이 살짝 비싼면이 있고 맛은 그냥 보통의 맛임 소스가 부족했음! 매운돼지김치어쩌...
8,Chaieun Park,6년 전,깔끔하고 맛있는 롤 도시락.\n다 맛있어 보여서 고르는데 어려웠는데\n맛이 다 비슷...
9,문진솔,5년 전,"깨끗하고 위생적이고, 간편하게 먹고 싶을 때 테이크아웃& 혼밥먹기 딱좋다! 다만 늦..."


In [8]:
reviews_df.to_csv('pascucci_reviews_google.csv', index=False, encoding='utf-8-sig')

In [13]:
import re 
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException, WebDriverException
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 구글 맵 페이지 열기 및 '뱅뱅막국수' 검색
driver.get("https://www.google.com/maps")
try:
    # 검색어 입력 및 검색
    search_box = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="searchboxinput"]'))
    )
    search_box.send_keys("스노우폭스 뱅뱅점")
    search_box.send_keys(Keys.ENTER)
except Exception as e:
    print(f"검색창을 찾을 수 없습니다: {e}")
    driver.quit()
    exit()

# 페이지 로드 대기
time.sleep(5)

try:
    # 첫 번째 검색 결과를 JavaScript로 클릭
    first_result = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '(//*[@class="hfpxzc"])[1]'))
    )
    driver.execute_script("arguments[0].click();", first_result)
except Exception as e:
    print(f"첫 번째 검색 결과를 클릭할 수 없습니다: {e}")
    
# 리뷰 페이지 열기
try:
    review_button = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.XPATH, '//button[contains(@aria-label, "리뷰 더보기")]'))
    )
    review_button.click()
    time.sleep(5)  # 리뷰 페이지가 완전히 로드되도록 추가 대기
except Exception as e:
    print(f"리뷰 페이지를 열 수 없습니다: {e}")
    driver.quit()
    exit()
    
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

# 리뷰 섹션의 컨테이너를 찾음
try:
    review_container = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div:nth-child(2) > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde'))
    ) #목록이 하나가 아닌경우: #QA0Szd > div > div > div.w6VYqd > div.bJzME.Hu9e2e.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde
    
    # 스크롤을 반복적으로 내리기
    max_wait_time = 5  # 최대 대기 시간 시작 (초)
    while True:
        # 현재 로드된 리뷰 수 확인
        reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')
        previous_count = len(reviews)
        
        # 스크롤을 아래로 내림
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", review_container)
        
        # 새로운 리뷰가 로드될 때까지 대기
        try:
            WebDriverWait(driver, max_wait_time).until(
                lambda driver: len(driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')) > previous_count
            )
            # 리뷰가 로드되면 대기 시간을 초기화
            max_wait_time = 5
        except TimeoutException:
            # 새로운 리뷰가 로드되지 않으면 더 이상 로드할 리뷰가 없는 것으로 간주하고 종료
            print("더 이상 로드할 리뷰가 없습니다.")
            break

        # 스크롤이 아래로 내려갈수록 로딩 시간이 길어질 수 있으므로, 실패 시 대기 시간 증가
        max_wait_time += 1  # 한 번 실패할 때마다 대기 시간을 1초씩 늘림 (필요 시 조정 가능)

    print("모든 리뷰를 끝까지 로드했습니다.")
except Exception as e:
    print(f"스크롤 중 오류 발생: {e}")

# 리뷰 데이터 수집
reviews_list = []
reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')
for review in reviews:
    try:
        # '자세히' 버튼 클릭
        try:
            more_button = review.find_element(By.XPATH, './/span[2]/button')
            more_button.click()
            time.sleep(1)  # '자세히' 클릭 후 리뷰 로드 대기
        except NoSuchElementException:
            pass  # '자세히' 버튼이 없으면 무시하고 진행
        
        # 리뷰 작성자 ID
        reviewer_id = review.find_element(By.CSS_SELECTOR, '.d4r55').text
        # 리뷰 작성 날짜
        date = review.find_element(By.CSS_SELECTOR, '.rsqaWe').text
        # 리뷰 내용
        review_text = review.find_element(By.CSS_SELECTOR, '.wiI7pd').text
        reviews_list.append({
            "ID": reviewer_id,
            "Date": date,
            "Review": review_text
        })
    except Exception as e:
        print(f"리뷰 수집 중 오류: {e}")
        continue

# 데이터프레임으로 변환 및 출력
reviews_df = pd.DataFrame(reviews_list)
reviews_df

첫 번째 검색 결과를 클릭할 수 없습니다: Message: 
Stacktrace:
	GetHandleVerifier [0x00BFEBD3+24307]
	(No symbol) [0x00B88D74]
	(No symbol) [0x00A6C323]
	(No symbol) [0x00AADC86]
	(No symbol) [0x00AADECB]
	(No symbol) [0x00AEB9F2]
	(No symbol) [0x00ACFED4]
	(No symbol) [0x00AE9579]
	(No symbol) [0x00ACFC26]
	(No symbol) [0x00AA219C]
	(No symbol) [0x00AA311D]
	GetHandleVerifier [0x00EA8D93+2818227]
	GetHandleVerifier [0x00F0542E+3196750]
	GetHandleVerifier [0x00EFD9D2+3165426]
	GetHandleVerifier [0x00C9DA70+675216]
	(No symbol) [0x00B91B3D]
	(No symbol) [0x00B8EA18]
	(No symbol) [0x00B8EBB5]
	(No symbol) [0x00B81640]
	BaseThreadInitThunk [0x758DFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x7787809E+286]
	RtlGetAppContainerNamedObjectPath [0x7787806E+238]

더 이상 로드할 리뷰가 없습니다.
모든 리뷰를 끝까지 로드했습니다.
리뷰 수집 중 오류: Message: no such element: Unable to locate element: {"method":"css selector","selector":".wiI7pd"}
  (Session info: chrome=130.0.6723.119); For documentation on this error, please visit: https://www.se

리뷰 수집 중 오류: Message: no such element: Unable to locate element: {"method":"css selector","selector":".wiI7pd"}
  (Session info: chrome=130.0.6723.119); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00BFEBD3+24307]
	(No symbol) [0x00B88D74]
	(No symbol) [0x00A6C323]
	(No symbol) [0x00AADC86]
	(No symbol) [0x00AADECB]
	(No symbol) [0x00AA3CF1]
	(No symbol) [0x00ACFED4]
	(No symbol) [0x00AA3C14]
	(No symbol) [0x00AD0124]
	(No symbol) [0x00AE9579]
	(No symbol) [0x00ACFC26]
	(No symbol) [0x00AA219C]
	(No symbol) [0x00AA311D]
	GetHandleVerifier [0x00EA8D93+2818227]
	GetHandleVerifier [0x00F0542E+3196750]
	GetHandleVerifier [0x00EFD9D2+3165426]
	GetHandleVerifier [0x00C9DA70+675216]
	(No symbol) [0x00B91B3D]
	(No symbol) [0x00B8EA18]
	(No symbol) [0x00B8EBB5]
	(No symbol) [0x00B81640]
	BaseThreadInitThunk [0x758DFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x778

리뷰 수집 중 오류: Message: no such element: Unable to locate element: {"method":"css selector","selector":".wiI7pd"}
  (Session info: chrome=130.0.6723.119); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00BFEBD3+24307]
	(No symbol) [0x00B88D74]
	(No symbol) [0x00A6C323]
	(No symbol) [0x00AADC86]
	(No symbol) [0x00AADECB]
	(No symbol) [0x00AA3CF1]
	(No symbol) [0x00ACFED4]
	(No symbol) [0x00AA3C14]
	(No symbol) [0x00AD0124]
	(No symbol) [0x00AE9579]
	(No symbol) [0x00ACFC26]
	(No symbol) [0x00AA219C]
	(No symbol) [0x00AA311D]
	GetHandleVerifier [0x00EA8D93+2818227]
	GetHandleVerifier [0x00F0542E+3196750]
	GetHandleVerifier [0x00EFD9D2+3165426]
	GetHandleVerifier [0x00C9DA70+675216]
	(No symbol) [0x00B91B3D]
	(No symbol) [0x00B8EA18]
	(No symbol) [0x00B8EBB5]
	(No symbol) [0x00B81640]
	BaseThreadInitThunk [0x758DFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x778

리뷰 수집 중 오류: Message: no such element: Unable to locate element: {"method":"css selector","selector":".wiI7pd"}
  (Session info: chrome=130.0.6723.119); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00BFEBD3+24307]
	(No symbol) [0x00B88D74]
	(No symbol) [0x00A6C323]
	(No symbol) [0x00AADC86]
	(No symbol) [0x00AADECB]
	(No symbol) [0x00AA3CF1]
	(No symbol) [0x00ACFED4]
	(No symbol) [0x00AA3C14]
	(No symbol) [0x00AD0124]
	(No symbol) [0x00AE9579]
	(No symbol) [0x00ACFC26]
	(No symbol) [0x00AA219C]
	(No symbol) [0x00AA311D]
	GetHandleVerifier [0x00EA8D93+2818227]
	GetHandleVerifier [0x00F0542E+3196750]
	GetHandleVerifier [0x00EFD9D2+3165426]
	GetHandleVerifier [0x00C9DA70+675216]
	(No symbol) [0x00B91B3D]
	(No symbol) [0x00B8EA18]
	(No symbol) [0x00B8EBB5]
	(No symbol) [0x00B81640]
	BaseThreadInitThunk [0x758DFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x778

리뷰 수집 중 오류: Message: no such element: Unable to locate element: {"method":"css selector","selector":".wiI7pd"}
  (Session info: chrome=130.0.6723.119); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00BFEBD3+24307]
	(No symbol) [0x00B88D74]
	(No symbol) [0x00A6C323]
	(No symbol) [0x00AADC86]
	(No symbol) [0x00AADECB]
	(No symbol) [0x00AA3CF1]
	(No symbol) [0x00ACFED4]
	(No symbol) [0x00AA3C14]
	(No symbol) [0x00AD0124]
	(No symbol) [0x00AE9579]
	(No symbol) [0x00ACFC26]
	(No symbol) [0x00AA219C]
	(No symbol) [0x00AA311D]
	GetHandleVerifier [0x00EA8D93+2818227]
	GetHandleVerifier [0x00F0542E+3196750]
	GetHandleVerifier [0x00EFD9D2+3165426]
	GetHandleVerifier [0x00C9DA70+675216]
	(No symbol) [0x00B91B3D]
	(No symbol) [0x00B8EA18]
	(No symbol) [0x00B8EBB5]
	(No symbol) [0x00B81640]
	BaseThreadInitThunk [0x758DFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x778

,ID,Date,Review
0,이정민,10달 전,●접근성\n뱅뱅사거리 버스정류장에서 3분 이내\n\n●주차\n건물 지하 이용\n진입...
1,이샤샤,1년 전,SNOW FOX(feat.장문녀)\n\n롤이 땡긴다싶어서 롤전문점을 찾아보다가 알게...
2,S K,6년 전,진작에 여기 평점을 봤어야 됬는데..\n\n맛 자체는 나쁘지 않다. 깔끔하고 도시락...
3,머니 바이블,5년 전,깔끔하고 괜찮아요. 도시락치곤 가격 좀 센데 재료 궐리티가 높다고 하니 가끔 먹기엔...
4,Ryung-Hwa Rhee,5년 전,가격이 좀 비싸지만 혼밥하기 좋은 밥집이에요\n건강한 메뉴들로 먹고 싶을때 자주 먹...
5,Irene “MJ” Kim,5년 전,가격은 어느 근사한 음식점만큼 비싸다. 하지만 청결한 매장에서 신선한 식재료로 조리...
6,Sophia Ryu,5년 전,가격은 좀 나가지만 맛은 좋아요. 혼자 와서 간단히 샐러드나 롤 먹고 가기 좋을 듯해요.
7,Minji Kim,3년 전,가격이 살짝 비싼면이 있고 맛은 그냥 보통의 맛임 소스가 부족했음! 매운돼지김치어쩌...
8,Chaieun Park,6년 전,깔끔하고 맛있는 롤 도시락.\n다 맛있어 보여서 고르는데 어려웠는데\n맛이 다 비슷...
9,문진솔,5년 전,"깨끗하고 위생적이고, 간편하게 먹고 싶을 때 테이크아웃& 혼밥먹기 딱좋다! 다만 늦..."


## ------------------자동화

In [21]:
import os
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException
import time

# 가게명 리스트
store_names = ["스노우폭스 뱅뱅점", "맥도날드 서초뱅뱅점", "큐뮬러스", "유니마카롱", "짜짜루", "아티제 강남유니온센터점",
    "어가람", "드시옹", "웰니스 쌀빵", "파리바게뜨 뱅뱅사거리점", "파라노이드 강남뱅뱅 카페", "아티제 뱅뱅사거리점",
    "만복회해산물 뱅뱅사거리점", "해머스미스커피 뱅뱅사거리점", "월참치", "쿡쿡쿡 뱅뱅사거리점", "파스타엔포크 강남역삼점",
    "고가네철판불백", "곰작골나주곰탕 역삼초교점", "사람사는고깃집김일도 뱅뱅점", "잇풀키토김밥 샌드위치 샐러드 강남점",
    "엄마손맛집밥세로방", "고불초쌈밥 2호점", "우심터", "난타5000", "봉구스밥버거 역삼삼일점", "세로방", "커피콩스토리", "더에이치",
    "9CAFE", "소뭉집 본점", "리첸", "지우네", "중화카츠", "교토일식", "회춘식당", "노브랜드버거 뱅뱅사거리점", "라망드쉐프", "서초골",
    "에슬로우커피 뱅뱅사거리점", "전주피순대",
"오로스트커피", "다슬음", "조순금닭도리탕", "궁민김밥", "더벤티 뱅뱅사거리점",
    "돼지벅스", "시원한대구탕", "판문점부대찌개 역삼점", "청담배짱이", "통돼지 두루치기김치찌개", "봉평착한메밀", "칸꼬시",
    "곰바로곰탕", "문화시민", "오늘은 어떤닭", "카페블랑131", "버드나무집 서초본점", "파스쿠찌 뱅뱅사거리점", "남강매점",
    "스타벅스 서초태우빌딩점", "맥켄치킨", "봉피양 양재점", "깡돈", "북경반점", "무화잠 강남점", "아워948", "참족", "예당", "다돈식당",
    "낙여삼", "명동할머니국수 뱅뱅사거리점", "이천쌀밥", "또봉이통닭 도곡점", "카린지린가네스낵바 뱅뱅사거리점", "왕산골",
    "서초남순남순대국 본점", "브라운돈까스 뱅뱅사거리점", "고품격커피공장 뱅뱅사거리점", "두두돼지불백", "사천루",
 "장수촌풍천장어",
    "뱅뱅막국수 역삼본점", "커피박스", "뚜레쥬르 뱅뱅사거리점", "멜론", "오아시스", "에이림커피", "카페S", "원할매이모네닭한마리",
    "20년전통영이네떡볶이", "영이네추억의떡볶이", "비채", "한강수", "고래똥", "두메 도곡점", "썸",
      "샵5827 에스프레소",
    "메종보탄"
]


# CSV 저장 경로 설정
output_folder = '200m이내_가게_리뷰_구글'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 가게별로 웹브라우저를 열고 리뷰 추출 후 닫기
for store_name in store_names:
    # ChromeDriver 초기화
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")
    options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
    options.add_argument("--disable-blink-features=AutomationControlled")
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

    # 구글 맵 페이지 열기
    driver.get("https://www.google.com/maps")

    try:
        # 검색어 입력 및 검색
        search_box = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="searchboxinput"]'))
        )
        search_box.send_keys(store_name)
        search_box.send_keys(Keys.ENTER)
    except Exception as e:
        print(f"{store_name}의 검색창을 찾을 수 없습니다: {e}")
        driver.quit()
        continue

    # 페이지 로드 대기
    time.sleep(6)

    try:
        # 첫 번째 검색 결과 클릭
        first_result = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '(//*[@class="hfpxzc"])[1]'))
        )
        driver.execute_script("arguments[0].click();", first_result)
    except Exception as e:
        print(f"{store_name}의 첫 번째 검색 결과를 클릭할 수 없습니다: {e}")
        #driver.quit()
        continue
    time.sleep(2)
    # 리뷰 페이지 열기
    try:
        review_button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(@aria-label, "리뷰 더보기")]'))
        )
        review_button.click()
        time.sleep(5)
    except Exception as e:
        print(f"{store_name}의 리뷰 페이지를 열 수 없습니다: {e}")
        driver.quit()
        continue

    # 리뷰 컨테이너 찾기 및 스크롤
    try:
        review_container = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div:nth-child(2) > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde'))
        )
        max_wait_time = 5
        while True:
            reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')
            previous_count = len(reviews)
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", review_container)
            try:
                WebDriverWait(driver, max_wait_time).until(
                    lambda driver: len(driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')) > previous_count
                )
                max_wait_time = 5
            except TimeoutException:
                print("더 이상 로드할 리뷰가 없습니다.")
                break
            max_wait_time += 1
        print(f"{store_name}의 모든 리뷰를 끝까지 로드했습니다.")
    except Exception as e:
        print(f"{store_name}의 스크롤 중 오류 발생: {e}")
        driver.quit()
        continue

    # 리뷰 데이터 수집
    reviews_list = []
    reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')
    for review in reviews:
        try:
            try:
                more_button = review.find_element(By.XPATH, './/span[2]/button')
                more_button.click()
                time.sleep(1)
            except NoSuchElementException:
                pass

            reviewer_id = review.find_element(By.CSS_SELECTOR, '.d4r55').text
            date = review.find_element(By.CSS_SELECTOR, '.rsqaWe').text
            review_text = review.find_element(By.CSS_SELECTOR, '.wiI7pd').text
            reviews_list.append({
                "ID": reviewer_id,
                "Date": date,
                "Review": review_text
            })
        except Exception as e:
            print(f"{store_name}의 리뷰 수집 중 오류: {e}")
            continue

    # 데이터프레임으로 저장 및 CSV 파일로 저장
    reviews_df = pd.DataFrame(reviews_list)
    file_name = os.path.join(output_folder, f"{store_name.replace(' ', '_')}_reviews_google.csv")
    reviews_df.to_csv(file_name, index=False, encoding="utf-8-sig")
    print(f"{file_name} 파일로 저장되었습니다.")

    # 브라우저 닫기
    driver.quit()

스노우폭스 뱅뱅점의 첫 번째 검색 결과를 클릭할 수 없습니다: Message: 
Stacktrace:
	GetHandleVerifier [0x00BFEBD3+24307]
	(No symbol) [0x00B88D74]
	(No symbol) [0x00A6C323]
	(No symbol) [0x00AADC86]
	(No symbol) [0x00AADECB]
	(No symbol) [0x00AEB9F2]
	(No symbol) [0x00ACFED4]
	(No symbol) [0x00AE9579]
	(No symbol) [0x00ACFC26]
	(No symbol) [0x00AA219C]
	(No symbol) [0x00AA311D]
	GetHandleVerifier [0x00EA8D93+2818227]
	GetHandleVerifier [0x00F0542E+3196750]
	GetHandleVerifier [0x00EFD9D2+3165426]
	GetHandleVerifier [0x00C9DA70+675216]
	(No symbol) [0x00B91B3D]
	(No symbol) [0x00B8EA18]
	(No symbol) [0x00B8EBB5]
	(No symbol) [0x00B81640]
	BaseThreadInitThunk [0x758DFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x7787809E+286]
	RtlGetAppContainerNamedObjectPath [0x7787806E+238]

맥도날드 서초뱅뱅점의 검색창을 찾을 수 없습니다: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=130.0.6723.119)
Stacktrace:
	GetHandleVerifier [0x00BFEBD3+24307]
	(No symbol) [0x00B88D74]
	(N

KeyboardInterrupt: 

In [20]:
import os
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import time

# 가게명 리스트
store_names = ["스노우폭스 뱅뱅점", "맥도날드 서초뱅뱅점", "큐뮬러스", "유니마카롱", "짜짜루", "아티제 강남유니온센터점",
    "어가람", "드시옹", "웰니스 쌀빵", "파리바게뜨 뱅뱅사거리점", "파라노이드 강남뱅뱅 카페", "아티제 뱅뱅사거리점",
    "만복회해산물 뱅뱅사거리점", "해머스미스커피 뱅뱅사거리점", "월참치", "쿡쿡쿡 뱅뱅사거리점", "파스타엔포크 강남역삼점",
    "고가네철판불백", "곰작골나주곰탕 역삼초교점", "사람사는고깃집김일도 뱅뱅점", "잇풀키토김밥 샌드위치 샐러드 강남점",
    "엄마손맛집밥세로방", "고불초쌈밥 2호점", "우심터", "난타5000", "봉구스밥버거 역삼삼일점", "세로방", "커피콩스토리", "더에이치",
    "9CAFE", "소뭉집 본점", "리첸", "지우네", "중화카츠", "교토일식", "회춘식당", "노브랜드버거 뱅뱅사거리점", "라망드쉐프", "서초골",
    "에슬로우커피 뱅뱅사거리점", "전주피순대",
"오로스트커피", "다슬음", "조순금닭도리탕", "궁민김밥", "더벤티 뱅뱅사거리점",
    "돼지벅스", "시원한대구탕", "판문점부대찌개 역삼점", "청담배짱이", "통돼지 두루치기김치찌개", "봉평착한메밀", "칸꼬시",
    "곰바로곰탕", "문화시민", "오늘은 어떤닭", "카페블랑131", "버드나무집 서초본점", "파스쿠찌 뱅뱅사거리점", "남강매점",
    "스타벅스 서초태우빌딩점", "맥켄치킨", "봉피양 양재점", "깡돈", "북경반점", "무화잠 강남점", "아워948", "참족", "예당", "다돈식당",
    "낙여삼", "명동할머니국수 뱅뱅사거리점", "이천쌀밥", "또봉이통닭 도곡점", "카린지린가네스낵바 뱅뱅사거리점", "왕산골",
    "서초남순남순대국 본점", "브라운돈까스 뱅뱅사거리점", "고품격커피공장 뱅뱅사거리점", "두두돼지불백", "사천루",
 "장수촌풍천장어",
    "뱅뱅막국수 역삼본점", "커피박스", "뚜레쥬르 뱅뱅사거리점", "멜론", "오아시스", "에이림커피", "카페S", "원할매이모네닭한마리",
    "20년전통영이네떡볶이", "영이네추억의떡볶이", "비채", "한강수", "고래똥", "두메 도곡점", "썸",
      "샵5827 에스프레소",
    "메종보탄"
]

# CSV 저장 경로 설정
output_folder = '200m이내_가게_리뷰_구글'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for store_name in store_names:
    # ChromeDriver 초기화
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")
    options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
    options.add_argument("--disable-blink-features=AutomationControlled")
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

    # 구글 맵 페이지 열기
    driver.get("https://www.google.com/maps")

    try:
        # 검색어 입력 및 검색
        search_box = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="searchboxinput"]'))
        )
        search_box.send_keys(store_name)
        search_box.send_keys(Keys.ENTER)
    except Exception as e:
        print(f"{store_name}의 검색창을 찾을 수 없습니다: {e}")
        driver.quit()
        continue

    # 페이지 로드 대기
    time.sleep(5)

    try:
        # 첫 번째 검색 결과 클릭
        first_result = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '(//*[@class="hfpxzc"])[1]'))
        )
        driver.execute_script("arguments[0].click();", first_result)
    except Exception as e:
        print(f"{store_name}의 첫 번째 검색 결과를 클릭할 수 없습니다: {e}")
        
        continue

    # 리뷰 페이지 열기
    try:
        review_button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(@aria-label, "리뷰 더보기")]'))
        )
        review_button.click()
        time.sleep(5)
    except Exception as e:
        print(f"{store_name}의 리뷰 페이지를 열 수 없습니다: {e}")
        driver.quit()
        continue
        
    

    # 리뷰 섹션의 컨테이너를 찾음 및 스크롤
    try:
        review_container = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div:nth-child(2) > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde'))
        )
        max_wait_time = 5
        while True:
            reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')
            previous_count = len(reviews)
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", review_container)
            try:
                WebDriverWait(driver, max_wait_time).until(
                    lambda driver: len(driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')) > previous_count
                )
                max_wait_time = 5
            except TimeoutException:
                print("더 이상 로드할 리뷰가 없습니다.")
                break
            max_wait_time += 1
        print(f"{store_name}의 모든 리뷰를 끝까지 로드했습니다.")
    except Exception as e:
        print(f"{store_name}의 스크롤 중 오류 발생: {e}")
        driver.quit()
        continue

    # 리뷰 데이터 수집
    reviews_list = []
    reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')
    for review in reviews:
        try:
            try:
                more_button = review.find_element(By.XPATH, './/span[2]/button')
                more_button.click()
                time.sleep(1)
            except NoSuchElementException:
                pass

            reviewer_id = review.find_element(By.CSS_SELECTOR, '.d4r55').text
            date = review.find_element(By.CSS_SELECTOR, '.rsqaWe').text
            review_text = review.find_element(By.CSS_SELECTOR, '.wiI7pd').text
            reviews_list.append({
                "ID": reviewer_id,
                "Date": date,
                "Review": review_text
            })
        except Exception as e:
            print(f"{store_name}의 리뷰 수집 중 오류: {e}")
            continue

    # 데이터프레임으로 저장 및 CSV 파일로 저장
    reviews_df = pd.DataFrame(reviews_list)
    file_name = os.path.join(output_folder, f"{store_name.replace(' ', '_')}_reviews_google.csv")
    reviews_df.to_csv(file_name, index=False, encoding="utf-8-sig")
    print(f"{file_name} 파일로 저장되었습니다.")

    # 브라우저 닫기
    driver.quit()


스노우폭스 뱅뱅점의 첫 번째 검색 결과를 클릭할 수 없습니다: Message: 
Stacktrace:
	GetHandleVerifier [0x00BFEBD3+24307]
	(No symbol) [0x00B88D74]
	(No symbol) [0x00A6C323]
	(No symbol) [0x00AADC86]
	(No symbol) [0x00AADECB]
	(No symbol) [0x00AEB9F2]
	(No symbol) [0x00ACFED4]
	(No symbol) [0x00AE9579]
	(No symbol) [0x00ACFC26]
	(No symbol) [0x00AA219C]
	(No symbol) [0x00AA311D]
	GetHandleVerifier [0x00EA8D93+2818227]
	GetHandleVerifier [0x00F0542E+3196750]
	GetHandleVerifier [0x00EFD9D2+3165426]
	GetHandleVerifier [0x00C9DA70+675216]
	(No symbol) [0x00B91B3D]
	(No symbol) [0x00B8EA18]
	(No symbol) [0x00B8EBB5]
	(No symbol) [0x00B81640]
	BaseThreadInitThunk [0x758DFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x7787809E+286]
	RtlGetAppContainerNamedObjectPath [0x7787806E+238]

맥도날드 서초뱅뱅점의 첫 번째 검색 결과를 클릭할 수 없습니다: Message: 
Stacktrace:
	GetHandleVerifier [0x00BFEBD3+24307]
	(No symbol) [0x00B88D74]
	(No symbol) [0x00A6C323]
	(No symbol) [0x00AADC86]
	(No symbol) [0x00AADECB]
	(No symbol) [0x00AEB9F2]
	(No symbol) [

KeyboardInterrupt: 